In [1]:
!pip install tensorflow torch transformers datasets nltk rouge-score scikit-learn

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=2fc180816c792c3329fb83fcb74f7d86a456746c4ee49eb34f947b6580383d11
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

In [ ]:
from datasets import load_dataset
from transformers import BartTokenizer

# Load the dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')

# Initialize the tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(examples['article'], max_length=1024, truncation=True)
    outputs = tokenizer(examples['highlights'], max_length=150, truncation=True)
    inputs['labels'] = outputs['input_ids']
    return inputs

# Apply preprocessing
processed_dataset = dataset.map(preprocess_function, batched=True)


In [ ]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

# Load the pre-trained BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    output_dir='./results',
    evaluation_strategy='epoch',
    logging_dir='./logs',
    num_train_epochs=3,
    save_total_limit=1,
    logging_steps=100,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['validation'],
)

# Train the model
trainer.train()


In [ ]:
from datasets import load_metric

rouge = load_metric('rouge')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = tokenizer.batch_decode(logits, skip_special_tokens=True)
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge.compute(predictions=predictions, references=references)

trainer.compute_metrics = compute_metrics
eval_results = trainer.evaluate()
print(eval_results)


In [ ]:
from datasets import load_metric

rouge = load_metric('rouge')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = tokenizer.batch_decode(logits, skip_special_tokens=True)
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge.compute(predictions=predictions, references=references)

trainer.compute_metrics = compute_metrics
eval_results = trainer.evaluate()
print(eval_results)
